# DE II A2 - Distributed Infrastructure, Model Serving and CI/CD

## Summary

The lab assignment covers the practical part of the discussed concepts of dynamic contextualization and model serving in a scalable production environment and reliable continuous integration and development process. The lab consists of four tasks and one optional task. First three tasks are compulsory, forth task is a bonus task and optional task does not have any points.\
实验作业涵盖了所讨论的dynamic contextualization概念的实践部分，以及在可伸缩生产环境和可靠的持续集成和开发过程中服务的模型。该实验室由四个任务和一个可选任务组成。前三个任务是强制性的，第四个任务是奖励任务，可选任务没有任何分数。

## Important:

1. Please terminate VMs once you finish the task.\
任务完成后，请立即终止虚拟机。
2. We recommend you to create a VM in SSC and execute all the tasks on that virtual machine. You can run the tasks on your laptops but it may break your local working environment.\
我们建议您在SSC中创建一个虚拟机，并在该虚拟机上执行所有任务。你可以在笔记本电脑上运行这些任务，但这可能会破坏你当地的工作环境。
3. For all the tasks, clone the repository: [model_serving](https://github.com/sztoor/model_serving.git)\
对于所有任务，克隆存储库：[model_serving](https://github.com/sztoor/model_serving.git)
4. The code for tasks is available in the `model_serving` directory.\
任务的代码可以在`model_serving`目录中找到。

```shell
- model_serving
    -- Single_server_without_docker
    -- Single_server_with_docker
    -- CI_CD
    -- OpenStack-Client
```

## Dynamic Contextualization

Dynamic contextualization is a process of preparing a customized computing environment at runtime. The process is ranging from creating/defining user roles and permissions to updating/installing different packages and initiating services.\
动态上下文化是在运行时准备定制计算环境的过程。这个过程从创建/定义用户角色和权限到更新/安装不同的包和启动服务。

### Task-1: Single Server Deployment

In this task, we will learn how the dynamic contextualization works using Cloud-Init package. For this task, we need to use OpenStack APIs to start a new VM and contextualize it at run time. The contextualization process will setup the following working environment:\
在本任务中，我们将学习动态上下文化是如何工作的，使用Cloud-Init包。对于这个任务，我们需要使用OpenStack APIs来启动一个新的VM，并在运行时将其上下文化。上下文化过程将设置以下工作环境：
1. Flask based web application as a frontend server\
基于Flask的web应用作为前端服务器
2. Celery and RabbitMQ server for backend server\
Celery和RabbitMQ服务器作为后端服务器
3. Model execution environment based on Keras and TensorFlow\
基于Keras和TensorFlow的模型执行环境

In case you are not familiar with the above-mentioned packages please read the following links:\
如果你不熟悉上述的包裹，请阅读以下链接：
1. Flask Application -> [Welcome to Flask — Flask Documentation (1.1.x)](https://flask.palletsprojects.com/en/1.1.x/)
2. Celery and RabbitMQ -> [Getting Started — Celery 5.0.5 documentation](https://docs.celeryproject.org/en/stable/getting-started/)
3. Keras and TensorFlow -> [Keras | TensorFlow Core](https://www.tensorflow.org/guide/keras)

The process will start when a client sends a new prediction request from the frontend web server. The server will pass the request to the backend Celery environment where running workers in the setup will pick up the task, run the predictions by loading the available model, send back the results and finally frontend server will display the results.\
当client从前端web服务器发送一个新的预测请求时，该进程将启动。服务器将把请求传递给后端Celery环境，在那里运行设置中的工作人员将捡起任务，通过加载可用模型运行预测，返回结果，最后前端服务器将显示结果。

**Steps for VM contextualization**

#### 1. Start a VM from the SSC dashboard and login.

* Log into the [SNIC cloud](https://east-1.cloud.snic.se/).
* Log in VM:
```shell
ssh -i /Users/lmf/PycharmProjects/MSc_DS/Important/key_DE.pem ubuntu@130.238.29.126
```
    * Remove host key in `~/.ssh/known_hosts` to get rid of message `WARNING: REMOTE HOST IDENTIFICATION HAS CHANGED!`.
    ```shell
    vim ~/.ssh/known_hosts
    ```

#### 2. Clone the git repository.

```shell
git clone https://github.com/sztoor/model_serving.git
```

#### 3. Go to the `model_serving/single_server_without_docker/production_server/` directory.

The directory contains the code that will run on the production server VM.\
该目录包含将在production服务器VM上运行的代码。

Following is the structure of the code:\
以下是代码的结构：
* Flask Application based frontend\
基于前端的Flask应用程序
    * `app.py`
    * `static`
    * `templates`
* Celery and RabbitMQ setup\
Celery和RabbitMQ设置
    * `run_task.py`
    * `workerA.py`
* Machine learning Model and Data\
机器学习模型和数据
    * `model.h5`
    * `model.json`
    * `pima-indians-diabetes.csv`

Open files and understand the application's structure.\
打开文件并理解应用程序的结构。

#### 4. Go to the `model_serving/openstack-client/single_node_without_docker_client/` directory.

This is the code that we will use to contextualize our production server.\
这是我们将用于上下文化production服务器的代码。

The code is based on the following two files:\
该代码基于以下两个文件:

* Cloud-Init configuration file\
Cloud-Init配置文件
    * `cloud-cfg.txt`
* OpenStack python code\
OpenStack python代码
    * `start_instance.py`

Open the files and understand the steps.\
打开文件并理解其中的步骤。
* __NOTE:__ You need to setup variable values in the `start_instance.py` script.\
您需要在`start_instance.py`脚本中设置变量值。
```shell
cd model_serving/openstack-client/single_node_without_docker_client/
vim start_instance.py
```
Modify `start_instance.py`:
```python
flavor = "ssc.medium"
private_net = "UPPMAX 2020/1-3 Internal IPv4 Network"
floating_ip_pool_name = None
floating_ip = None
image_name = "98c10a7f-2587-450b-866c-1266ea0dbe4b"
```
    * https://docs.openstack.org/python-novaclient/ocata/ref/v2/servers.html
* __Important:__ In order to run this code, you need to have OpenStack API environment running.\
运行此代码需要运行OpenStack API环境。
* __NOTE:__ Openstack APIs are only need to be installed on the client VM.\
Openstack API只需要安装在client VM上。

#### 5. Follow the instructions available on the following links:

* [OpenStack packages for Ubuntu](https://docs.openstack.org/install-guide/environment-packages-ubuntu.html)
```shell
# Enable the repository for Ubuntu Cloud Archive
sudo add-apt-repository cloud-archive:victoria
# Finalize the installation
# 1. Upgrade packages on all nodes:
sudo -s
sudo apt update && apt dist-upgrade
# 2. Install the OpenStack client:
sudo apt install python3-openstackclient
```

* [Install the OpenStack command-line clients](http://docs.openstack.org/mitaka/user-guide/common/cli_install_openstack_command_line_clients.html)

Download the client tools and API for OpenStack.\
下载OpenStack的客户端工具和API。

Download the Runtime Configuration (RC) file (version 3) from the SSC site (Top left frame, `Project` -> `API Access` -> `Download OpenStack RC File`).\
从SSC站点下载Runtime Configuration（RC）文件（版本3）（左上角，`项目` -> `访问API` -> `下载OpenStack RC文件`）。
```shell
vim DEII_L2_T1-openrc.sh
```

Set API access password. Go to [SNIC Science Cloud (SSC)](https://cloud.snic.se/), Left frame, under `Services` `Set your API password`.\
设置API访问密码。转到[SNIC Science Cloud (SSC)](https://cloud.snic.se/)，左边，在`SERVICES`下，`Set your API password`。

Confirm that your RC file have following environment variables:\
确认你的RC文件有以下环境变量：
```shell
export OS_USER_DOMAIN_NAME="snic"
export OS_IDENTITY_API_VERSION="3"
export OS_PROJECT_DOMAIN_NAME="snic"
export OS_PROJECT_NAME="UPPMAX 2020/1-3"
```

#### 6. Set the environment variables by sourcing the RC-file in the client VM.
通过在client VM中source RC文件来设置环境变量。

```shell
# source <project_name>_openrc.sh
source DEII_L2_T1-openrc.sh
```
__NOTE:__ You need to enter the API access password.\
需要输入API访问密码。

The successful execution of the following commands will confirm that you have the correct packages available on your client VM:\
成功执行以下命令将确认您的client VM上有可用的正确软件包:
```shell
openstack server list
openstack image list
```

For the API communication, we need following extra packages:\
对于API通信，我们需要以下额外的包:
```shell
sudo apt install python3-openstackclient
sudo apt install python3-novaclient
sudo apt install python3-keystoneclient
```
__NOTE:__ You need to setup variable values in the `start_instance.py` script.\
您需要在`start_instance.py`脚本中设置变量值。

#### 7. Once you setup the environment, run the following command.
设置好环境后，运行以下命令。

```shell
cd model_serving/openstack-client/single_node_without_docker_client/
python3 start_instance.py
```

Output:
```shell
user authorization completed.
Creating instance ... 
waiting for 10 seconds.. 
Instance: prod_server_without_docker_2930 is in BUILD state, sleeping for 5 seconds more...
Instance: prod_server_without_docker_2930 is in ACTIVEstate
```

The command will start a new server and initiate the contextualization process. It will take approximately 10 to 15 minutes. The progress can be seen on the cloud dashboard. Once the process finish, attach a floating IP to your production server and access the webpage from your client machine.\
该命令将启动一个新服务器并启动contextualization过程。大约需要10到15分钟。可以在云dashboard上看到进展。处理完成后，将一个浮动IP附加到生产服务器上，并从client机器访问该网页。

Welcome page:
* `http://<PRODUCTION-SERVER-IP>:5100`
* http://130.238.28.163:5100

Predictions page:
* `http://<PRODUCTION-SERVER-IP>:5100/predictions`
* http://130.238.28.163:5100/predictions

***TERMINATE THE PRODUCTION SERVER VM STARTED FOR TASK-1!***

#### Questions

##### 1. Explain how the application works? Write a short paragraph about the framework.
解释应用程序如何工作？写一段简短的框架。

**Answer**

In the client server, Cloud-Init configuration file `cloud-cfg.txt` and OpenStack python code `start_instance.py` will be used to initiate a new production server and contextualize it at run time. Production server hosts the complete application and provide a web interface to access the results.

##### 2. What are the drawbacks of the contextualization strategy adopted in the task-1? Write at least four drawbacks.
在task-1中采用的上下文化策略的缺点是什么？至少写出四个缺点。

* [A Practical Guide to Choosing between Docker Containers and VMs](https://www.weave.works/blog/a-practical-guide-to-choosing-between-docker-containers-and-vms)
* [Are Containers Replacing Virtual Machines?](https://www.docker.com/blog/containers-replacing-virtual-machines/)
* [Docker Containers vs. Virtual Machines](https://www.aquasec.com/cloud-native-academy/docker-container/docker-containers-vs-virtual-machines/)
* [Docker vs. Virtual Machine: Differences You Need to Know](https://www.simplilearn.com/tutorials/docker-tutorial/docker-vs-virtual-machine)
* [How is Docker different from a virtual machine?](https://stackoverflow.com/questions/16047306/how-is-docker-different-from-a-virtual-machine)
* [What’s the Diff: VMs vs Containers](https://www.backblaze.com/blog/vm-vs-containers/)

**Answer**

* Heavyweight. Each VM runs not just a full copy of an operating system, but a virtual copy of all the hardware that the operating system needs to run, which quickly adds up to a lot of RAM and CPU cycles.

* Resource consuming. Since VM uses a separate OS; it causes more resources to be used.

* Slow provisioning. To deploy a single application, Virtual Machines need to start the entire OS, which would cause a full boot process

* Lack of portability. While transferring files, VMs should have a copy of the OS and its dependencies because of which image size is increased and becomes a tedious process to share data.

##### 3. Currently, the contextualization process takes 10 to 15 minutes. How can we reduce the deployment time?
目前，contextualization过程需要10到15分钟。我们如何减少部署时间?

* [A Practical Guide to Choosing between Docker Containers and VMs](https://www.weave.works/blog/a-practical-guide-to-choosing-between-docker-containers-and-vms)

**Answer**

Docker containers typically start in a few seconds or less, whereas virtual machines can take minutes. Thus, workloads that need to start very quickly, or that involve spinning apps up and down constantly, may be a good fit for Docker.

##### 4. Write half a page summary about the task and add screenshots if needed.
写半页的任务总结，如果需要的话添加截图。

**Answer**

Run command `python3 start_instance.py` to start a new server and initiate the contextualization process.

### Task-2: Single Server Deployment with Docker Containers

In this task, we will repeat the same deployment process but with Docker containers. This time we will create a flexible containerized deployment environment where each container has a defined role.\
在本任务中，我们将重复相同的部署过程，但使用的是Docker容器。这一次，我们将创建一个灵活的容器化部署环境，其中每个容器都有一个已定义的角色。

#### 1. Go to `model_serving/single_server_with_docker/production_server/` directory on the client VM. 

The directory contains the code that will run on your production server.\
该目录包含将在production服务器上运行的代码。

Following is the structure of the code:
* Flask Application based frontend
    * `app.py`
    * `static`
    * `templates`
* Celery and RabbitMQ setup
    * `run_task.py`
    * `workerA.py`
* Machine learning Model and Data
    * `model.h5`
    * `model.json`
    * `pima-indians-diabetes.csv`
* Docker files
    * `Dockerfile`
    * `docker-compose.yml`

Open files and understand the application's structure.\
打开文件并理解应用程序的结构。

#### 2. Go to the `model_serving/openstack-client/single_node_with_docker_client/` directory.

The directory contains the code that we will use to contextualize the production server.\
该目录包含我们将用于将production服务器上下文化的代码。

The code is based on the following two files:
* Cloud-Init configuration file
    * `cloud-cfg.txt`
* OpenStack python code
    * `start_instance.py`

Open the files and understand the steps. You will need to update the `key_name` value in `start_instance.py` file with a previously created keypair.\
打开文件并理解其中的步骤。您需要使用先前创建的密钥对更新`start_instance.py`文件中的`key_name`值。
```shell
cd model_serving/openstack-client/single_node_with_docker_client/
vim start_instance.py
```

__NOTE:__ You need to setup variable values in the `start_instance.py` script.

#### 3. Run the following command.

```shell
# Set the environment variables by sourcing the RC-file in the client VM.
source DEII_L2_T1-openrc.sh
cd model_serving/openstack-client/single_node_with_docker_client/
python3 start_instance.py
```

Output:
```shell
user authorization completed.
Creating instance ... 
waiting for 10 seconds.. 
Instance: prod_server_with_docker_4964 is in BUILD state, sleeping for 5 seconds more...
Instance: prod_server_with_docker_4964 is in ACTIVE state
```

The command will start a new server and initiate the contextualization process. It will take approximately 10 to 15 minutes. The progress can be seen on the cloud dashboard. Once the process finish, attach a floating IP to your production server and access the webpage from your client machine.\
该命令将启动一个新服务器并启动contextualization过程。大约需要10到15分钟。可以在云仪表板上看到进展。处理完成后，将一个浮动IP附加到production服务器上，并从client机器访问该网页。

Welcome page: 
* `http://<PRODUCTION-SERVER-IP>:5100`
* http://130.238.28.163:5100

Predictions page:
* `http://<PRODUCTION-SERVER-IP>:5100/predictions`
* http://130.238.28.163:5100/predictions

#### 4. The next step is to test the horizontal scalability of the setup.
下一步是测试设置的水平可伸缩性。

* Login to the production server.
```shell
# ssh -i <PRIVATE KEY> ubuntu@<PRODUCTION-SERVER-IP>
ssh -i /Users/lmf/PycharmProjects/MSc_DS/Important/key_DE.pem ubuntu@130.238.28.163
```

* Switch to the superuser mode.
```shell
sudo bash
```

* Check the cluster status:
```shell
cd /model_serving/single_server_with_docker/production_server
docker-compose ps
```
Following three containers are running on the production server:\
以下三个容器正在生产服务器上运行：
    * `production_server_rabbit_1` -> RabbitMQ server
    * `production_server_web_1` -> Flask based web application
    * `production_server_worker_1_1` -> Celery worker

* Now we will add multiple workers in the cluster using docker commands.\
现在我们将使用docker命令在集群中添加多个worker。
    * Currently, there is one worker available. We will add two more workers.\
    目前，只有一个worker可用。我们将再增加两名worker。
    ```shell
    docker-compose up --scale worker_1=3 -d
    ```
    Output:
    ```shell
    production_server_rabbit_1 is up-to-date
    Starting production_server_worker_1_1 ... 
    Starting production_server_worker_1_1 ... done
    Creating production_server_worker_1_2 ... done
    Creating production_server_worker_1_3 ... done
    ```
    
    * Check the cluster status.\
    检查集群状态。
    ```shell
    docker-compose ps
    ```
    Now we have 3 workers running in the system.\
    现在系统中有3个worker在运行。
    
    * Scale down the cluster.\
    缩小集群规模。
    ```shell
    docker-compose up --scale worker_1=1 -d
    ```
    Output:
    ```shell
    production_server_rabbit_1 is up-to-date
    Stopping and removing production_server_worker_1_2 ... 
    Stopping and removing production_server_worker_1_2 ... done
    Stopping and removing production_server_worker_1_3 ... done
    ```

***TERMINATE THE SERVER VM STARTED FOR TASK-2!***

#### Questions

##### 1. What problems Task-2 deployment strategy solves compared to the strategy adopted in Task-1?
与Task-1相比，Task-2的部署策略解决了哪些问题?

* [Why you should use Docker and containers](https://www.infoworld.com/article/3310941/why-you-should-use-docker-and-containers.html)

**Answer**

Docker enables more efficient use of system resources. Instances of containerized apps use far less memory than virtual machines, they start up and stop more quickly, and they can be packed far more densely on their host hardware.\
Docker可以更有效地利用系统资源。与虚拟机相比，容器化的应用实例使用的内存要少得多，它们启动和停止的速度更快，而且它们可以在主机硬件上更密集地打包。

Docker enables application portability. Because Docker containers encapsulate everything an application needs to run (and only those things), they allow applications to be shuttled easily between environments.\
Docker支持应用程序的可移植性。因为Docker容器封装了应用程序需要运行的所有东西(而且只封装了那些东西)，所以它们允许应用程序轻松地在不同环境之间穿梭。

##### 2. What are the outstanding issues that the deployment strategy of Task-2 cannot not address?
Task-2的部署策略不能解决的突出问题有哪些？

##### 3. What are the possible solutions to address those outstanding issues?
解决这些突出问题的可能解决办法是什么？

* [When Not to Use Docker: Understanding the Limitations of Containers](https://www.channelfutures.com/open-source/when-not-to-use-docker-understanding-the-limitations-of-containers)

**Answer**

* Security problem. Docker can improve security in some ways by isolating applications from the host system and from each other. Yet Docker also creates new security challenges — such as the difficulty of monitoring so many moving pieces within a dynamic, large-scale Docker environment. Running your processes inside the containers as a non-privileged user cannot guarantee security. It depends on the capabilities you add or remove. To mitigate the risks of Docker container breakout, you should not download ready-to-use containers from untrusted sources.

* Lack of cross-platform compatibility. An application designed to run in a Docker container on Windows can’t run on Linux, and vice versa. In highly heterogeneous environments composed of both both Windows and Linux servers, this makes Docker less attractive. Sometimes, it is easier to set up a server if you have several static apps.

* Data storage. By design, all Docker files are created inside a container and stored on a writable container layer. It may be difficult to retrieve the data out of the container if a different process needs it. Also, the writable layer of a container is connected to the host machine which the container is running on. If you need to move the data elsewhere, you cannot do it easily. More than that, all the data stored inside a container will be lost forever once the container shuts down. You have to think of ways to save your data somewhere else first. To keep data safe in Docker, you need to employ an additional tool – Docker Data Volumes. Yet, this solution is still quite clumsy and needs to be improved.

##### 4. What is the difference between horizontal and vertical scalability? Is the strategy adopted in Task-2 follow horizontal or vertical scalability?
水平可伸缩性和垂直可伸缩性之间的区别是什么？Task-2中采用的策略遵循的是水平可伸缩性还是垂直可伸缩性？

* [Scaling Horizontally vs. Scaling Vertically](https://www.section.io/blog/scaling-horizontally-vs-vertically/)

**Answer**

Horizontal scaling means scaling by adding more machines to your pool of resources, whereas vertical scaling refers to scaling by adding more power (e.g. CPU, RAM) to an existing machine. The strategy adopted in Task-2 follow horizontal scalability, since it adds more workers.

##### 5. Write half a page summary about the task and add screenshots if needed.
写半页的任务总结，如果需要的话添加截图。

**Answer**

The deployment process is similar to Task-1.

Check the cluster status with the command `docker-compose ps`.

Add 2 more workers in the cluster using docker commands.

## Continuous Integration and Continuous Delivery (CI/CD)

The next two tasks will cover scalable cluster deployments using Ansible playbooks and CI/CD environment using versioning system Git and an extension called Git Hooks.\
接下来的两个任务将涉及使用Ansible playbooks，和使用版本控制系统Git和名为Git Hooks的扩展的CI/CD环境的可扩展集群部署。

### Task-3: Deployment of multiple servers using Ansible

For this task we need a setup based on following three VMs:
* Client VM: This machine will serve as an Ansible host name and initiate the configuration process.\
这台机器将作为一个Ansible host名，并启动配置过程。
* Production Server: The machine will host the dockerised version of the application discussed in task-1.\
这台机器将托管task-1中讨论的应用程序的dockerised版本。
* Development Server: The machine will host the development environment and push the new changes to the production server.\
这台机器将承载开发环境，并将新更改推送到production服务器。

#### 1. Steps to setup Ansible host and orchestration environment
安装Ansible host和编配环境的步骤

If you are not familiar with the Ansible, visit following URLs:
* [Ansible is Simple IT Automation](https://www.ansible.com/)
* [The Ansible Basics: Why Automation Matters Today in IT](https://www.ansible.com/blog/it-automation)
* [OVERVIEW - How Ansible Works](https://www.ansible.com/overview/how-ansible-works)

#### 2. Login to the Client machine

```shell
# ssh -i <PRIVATE KEY> ubuntu@<PRODUCTION-SERVER-IP>
```

#### 3. Go to `model_serving/ci_cd` directory. 

This directory contains the following two subdirectories\
此目录包含以下两个子目录

`production_server`
* Flask Application based frontend
    * `app.py`
    * `static`
    * `templates`
* Celery and RabbitMQ setup
    * `run_task.py`
    * `workerA.py`
* Machine learning Model and Data
    * `model.h5`
    * `model.json`
    * `pima-indians-diabetes.csv`

`development_server`
* Machine learning Model and Data
    * `model.h5`
    * `model.json`
    * `pima-indians-diabetes.csv`
    * `neural_net.py`

Open files and understand the application's structure.\
打开文件并理解应用程序的结构。

#### 4. Go to `model_serving/openstack-client/single_node_with_docker_ansible_client` directory. 

The files in the directory will be used to contextualize the production and deployment servers.\
目录中的文件将被用来使production服务器和deployment服务器contextualize。

* CloudInit files
    * `prod-cloud-cfg.txt`
    * `dev-cloud-cfg.txt`
* OpenStack code
    * `start_instance.py`
* Ansible files
    * `setup_var.yml`
    * `configuration.yml`

The client code will start two VMs and by using Ansible orchestration environment. It will contextualize both of the VMs simultaneously.\
client代码将通过使用Ansible orchestration环境启动两个VM。它将同时contextualize这两个VM。

#### 5. Install and configure Ansible on the client machine.
在client机器上安装和配置Ansible。

* First generate a cluster SSH key.\
首先生成一个集群SSH密钥。

* Check the username you are login as\
检查您登录时使用的用户名
```shell
whoami
```

__Important:__ You need to be as ubuntu user. If you are root user switch back to ubuntu user.\
你必须是ubuntu用户。如果你是root用户切换回ubuntu用户。

Create a directory\
创建一个目录
```shell
mkdir -p /home/ubuntu/cluster-keys
ssh-keygen -t rsa
```
Set the file path `/home/ubuntu/cluster-keys/cluster-key`. Do not set the password, simply press Enter twice.\
设置文件路径为`/home/ubuntu/cluster-keys/cluster-key`。无需设置密码，只需按两次“Enter”即可。

```shell
Enter file in which to save the key (/home/ubuntu/.ssh/id_rsa): /home/ubuntu/cluster-keys/cluster-key
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /home/ubuntu/cluster-keys/cluster-key
Your public key has been saved in /home/ubuntu/cluster-keys/cluster-key.pub
The key fingerprint is:
SHA256:KJOudYc7hBjrABiL6grt8j3p3Y3CCxDKdAJcJiX6ir4 ubuntu@mengfeiliang-l2
The key's randomart image is:
+---[RSA 3072]----+
|ooo+             |
|+.+              |
|+=..             |
|Bo+. . .         |
|+.o++.. S        |
|+ooo.o..         |
|=o. ++o .        |
|+o.+o+++ o       |
|oE=.o.++o .      |
+----[SHA256]-----+
```

* The step will generate cluster ssh keys at the following location:\
该步骤将在以下位置生成集群ssh密钥:
    * Private key: `/home/ubuntu/cluster-keys/cluster-key`
    * Public key: `/home/ubuntu/cluster-keys/cluster-key.pub`

#### 6. Next, we will start the Production and Development servers.
接下来，我们将启动Production和Development服务器。

* Go to the `model_serving/openstack-client/single_node_with_docker_ansible_client`, open `prod-cloud-cfg.txt` delete the old key from the section `ssh_authorized_keys:` and copy the complete contents of `/home/ubuntu/cluster-keys/cluster-key.pub` in the `prod-cloud-cfg.txt` file.\
进入`model_serving/openstack-client/single_node_with_docker_ansible_client`，打开`prod-cloud-cfg.txt`从`ssh_authorized_keys:`小节中删除旧的密钥，并复制`/home/ubuntu/cluster-keys/cluster-key.pub`的完整内容到`prod-cloud-cfg.txt`文件中。
```shell
vim /home/ubuntu/cluster-keys/cluster-key.pub
```
client服务器上的集群公钥：
```shell
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQC9jz5EfAQ8rgj/gNZpy0DU+LB4d6v/QUbb4uni9r+J190DW401PhvF2zQPzLnPfbSJSv+S/Qf9oL/vNwigOBYrV9Bpr9JFfNCbdbZx+nm7hKVe1Hjmk82o4gMtvmvmjudWIUmRVodXAyWkBzTevAKK6QJUndz11Ptjlp3Cgr4s9Dr5hbTesQFKF886QgG0i7Kr+HB13mqxFq936hxRgj+MVjzA9hB0aLjsD1GklDBLBV1bCZAR+v1mucSASkGcWpvGtsnUPZMqVtccOiBCChWELN+5VyNbzt/J9AAUW4aoOAqKhWplETicjR8ST9efNkfNaMhyGrLRNetZCSvg59+l7DEt0d+Vf/45hYeFQUrY7Se5ZylQB8yVHl4xfX77sxGiMIU8FcVBtLLIJFYI2NTJgmtRi6BVoLMGC1Q9IyTPbbsahk+ZgUHrFZwLbGtBKevQysIh+M6l7zDaJ2aedWRMYBMu/WRISM2oGDHn5GyfUqOe97tejBaw5wSbXWXVSI8= ubuntu@mengfeiliang-l2
```
```shell
cd model_serving/openstack-client/single_node_with_docker_ansible_client
vim prod-cloud-cfg.txt
```

* Repeat same step 3 with the `dev-cloud-cfg.txt`. Delete the old key from the section `ssh_authorized_keys:` and copy the complete contents of `/home/ubuntu/clusterkeys/cluster-key.pub` in the `dev-cloud-cfg.txt` file.\
使用`dev-cloud-cfg.txt`重复相同的步骤3。删除`ssh_authorized_keys:`小节中的旧键，并复制`/home/ubuntu/clusterkeys/cluster-key.pub`的完整内容到在`dev-cloud-cfg.txt`文件中。
```shell
cd model_serving/openstack-client/single_node_with_docker_ansible_client
vim dev-cloud-cfg.txt
```

#### 7. Run the `start_instances.py` code.

```shell
source DEII_L2_T1-openrc.sh
cd model_serving/openstack-client/single_node_with_docker_ansible_client
vim start_instances.py
python3 start_instances.py
```

The output will give you the internal IPs of the VMs.
```shell
user authorization completed.
Creating instances ... 
waiting for 10 seconds.. 
Instance: prod_server_with_docker_2638 is in BUILD state, sleeping for 5 seconds more...
Instance: dev_server_2638 is in BUILD state, sleeping for 5 seconds more...
Instance: prod_server_with_docker_2638 is in ACTIVE state ip address: 192.168.2.198
Instance: dev_server_2638 is in ACTIVE state ip address: 192.168.2.80
```

Now we have two VM running with the internal IP addresses.

#### 8. Install Ansible packages on the client machine.
在client机器上安装Ansible包

```shell
sudo bash
apt update; apt upgrade
apt-add-repository ppa:ansible/ansible
apt update
apt install ansible
```

#### 9. Next step is to enter these IP addresses in the Ansible hosts file.
下一步是在Ansible hosts文件中输入这些IP地址。

Open the Ansible inventory file and add the IP addresses in that file.\
打开Ansible目录文件，并在该文件中添加IP地址。

For this step you need to swtich to root user.\
对于这个步骤，您需要切换到root用户。
```shell
sudo bash
nano /etc/ansible/hosts
```

File contents:\
文件内容：
```shell
[servers]
# prodserver ansible_host=<production server IP address>
# devserver ansible_host=<development server IP address>
prodserver ansible_host=192.168.2.198
devserver ansible_host=192.168.2.80

[all:vars]
ansible_python_interpreter=/usr/bin/python3

[prod_server]
prodserver ansible_connection=ssh ansible_user=appuser

[dev_server]
devserver ansible_connection=ssh ansible_user=appuser
```

If you need to learn more about Ansible, here are some useful links:\
如果你想了解更多关于Ansible的信息，这里有一些有用的链接:\
Easy installation instructions [How to Install and Configure Ansible on Ubuntu 18.04](https://www.digitalocean.com/community/tutorials/how-to-install-and-configure-ansible-on-ubuntu-18-04)

* Just to confirm the access permission are correctly set, access production and development server from the client VM.\
为了确认访问权限设置正确，请从client VM访问production和development服务器。

First switch back to user ubuntu\
首先切换回用户ubuntu
```shell
# ssh -i /home/ubuntu/cluster-keys/cluster-key appuser@<PRODUCTION-SERVER-IP>
ssh -i /home/ubuntu/cluster-keys/cluster-key appuser@192.168.2.198
```
If the login is successful, exit from the production server. Repeat the step with development server.\
如果登录成功，请退出production服务器。在development服务器上重复此步骤。
```shell
# ssh -i /home/ubuntu/cluster-keys/cluster-key appuser@<DEVELOPMENT-SERVER-IP>
ssh -i /home/ubuntu/cluster-keys/cluster-key appuser@192.168.2.80
```
If the login is successful, exit from the development server.\
如果登录成功，请退出开发服务器。

For this step, switch to ubuntu user on the client VM.\
该步骤需要在client虚拟机上切换为ubuntu用户。

Now we will run the Ansible script available in the `model_serving/openstack-client/single_node_with_docker_ansible_client` directory.\
现在我们将运行`model_serving/openstack-client/single_node_with_docker_ansible_client`目录中的Ansible脚本。
```shell
export ANSIBLE_HOST_KEY_CHECKING=False
ansible-playbook configuration.yml --private-key=/home/ubuntu/cluster-keys/cluster-key
```

The process will take 10 to 15 minutes to complete.\
这个过程需要10到15分钟才能完成。

Attach floating IP address to the production server and access the same predictions webpage as we did in previous tasks.\
将浮动IP地址附加到production服务器上，并访问我们在前面的任务中所做的相同的预测网页。

Welcome page: 
* `http://<PRODUCTION-SERVER-IP>:5100`
* http://130.238.28.163:5100

Predictions page:
* `http://<PRODUCTION-SERVER-IP>:5100/predictions`
* http://130.238.28.163:5100/predictions

***Important: Task-4 is the continuation of Task-3. Do NOT terminate your instances setup in Task-3.***

#### Questions

##### 1. What is the difference between CloudInit and Ansible?
CloudInit和Ansible的区别是什么?

**Answer**

Cloud-Init is the industry standard multi-distribution method for cross-platform cloud instance initialization. It is supported across all major public cloud providers, provisioning systems for private cloud infrastructure, and bare-metal installations.\
Cloud-Init是用于跨平台云实例初始化的行业标准多分发方法。所有主要的公共云提供商、私有云基础设施的供应系统和裸金属安装都支持它。

Ansible is a radically simple IT automation engine that automates cloud provisioning, configuration management, application deployment, intra-service orchestration, and many other IT needs.\
Ansible是一个非常简单的IT自动化引擎，它可以自动化云供应、配置管理、应用程序部署、服务内部编排和许多其他IT需求。

##### 2. Explain the configurations available in `dev-cloud-cfg.txt` and `prod-cloud-cfg.txt` files.
解释`dev-cloud-cfg.txt`和`prod-cloud-cfg.txt`文件中可用的配置。

* [How To Use Cloud-Config For Your Initial Server Setup](https://www.digitalocean.com/community/tutorials/how-to-use-cloud-config-for-your-initial-server-setup)
* [An Introduction to Cloud-Config Scripting](https://www.digitalocean.com/community/tutorials/an-introduction-to-cloud-config-scripting)

**Answer**

To create a new user, we use the `users` directive. 

`sudo: ALL=(ALL) NOPASSWD:ALL` configures password-less `sudo` access.

`home` directive is used to set the local path you want to use.

Default `/bin/sh` shell is a very basic environment, so we want to manually specify a `bash` shell environment for our new user, which can be accomplished with the `shell` directive.

In order to log into this new account without a password, we will need to provide one or more of our SSH public keys, which can be accomplished with the `ssh-authorized-keys` directive.

##### 3. What problem we have solved by using Ansible?
我们使用Ansible解决了什么问题?

* [Ansible | Ansible Use Cases](https://www.ansible.com/use-cases)

**Answer**

* Orchestration. Today’s IT brings complex deployments and complex challenges. Ansible can make it easier to orchestrate the complex tasks such as dealing with clustered applications, multiple datacenters, public, private and hybrid clouds and applications with complex dependencies.

* Continuous delivery. Ansible Playbooks keep the applications properly deployed (and managed) throughout their entire lifecycle.

* Configuration management. Ansible centralizes configuration file management and deployment.

### Task-4: CI/CD using Git HOOKS

In this task, we will build a reliable execution pipeline using Git Hooks. The pipeline will allow continuous integration and delivery of new machine learning models in a production environment.\
在这个任务中，我们将使用Git Hooks构建一个可靠的执行管道。该流水线将允许在生产环境中持续集成和交付新的机器学习模型。

#### 1. Enable SSH key based communication between production and development servers
在production服务器和development服务器之间启用基于SSH密钥的通信

* Login to the development server\
登录到development服务器
```shell
ssh -i /Users/lmf/PycharmProjects/MSc_DS/Important/key_DE.pem ubuntu@130.238.29.126
# ssh -i cluster-key appuser@<DEVELOPMENT-SERVER-IP>
ssh -i /home/ubuntu/cluster-keys/cluster-key appuser@192.168.2.80
```

* Generate SSH key\
生成SSH密钥
```shell
ssh-keygen
```
Output:

```shell
Generating public/private rsa key pair.
Enter file in which to save the key (/home/appuser/.ssh/id_rsa): 
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /home/appuser/.ssh/id_rsa
Your public key has been saved in /home/appuser/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:CGj5iz+y94LmoqoB5I0jXUPWD79EhVAJumDV3bo1TtY appuser@devserver
The key's randomart image is:
+---[RSA 3072]----+
|    +oo=.=.      |
|   * .+ = .      |
| .* =  = . .     |
|o+o+ + .= = E    |
|o+..o ..SB .     |
|o .. .  o .      |
|. ...            |
|..+.+            |
|B+o+.+.          |
+----[SHA256]-----+
```

__Important:__ Do not change the default `/home/appuser/.ssh/id_rsa` path of the file.\
不要更改文件的默认`/home/appuser/.ssh/id_rsa`路径。

Do not set the password, simply press Enter twice.\
无需设置密码，只需按两次“Enter”即可。

__NOTE:__ This step will create two files, private key `/home/appuser/.ssh/id_rsa` and public key `/home/appuser/.ssh/id_rsa.pub`.\
这一步将创建两个文件，私钥`/home/appuser/.ssh/id_rsa`和公钥`/home/appuser/.ssh/id_rsa.pub`。

* Copy the contents of public key file `/home/appuser/.ssh/id_rsa.pub` from the development server.\
从development服务器拷贝公钥文件`/home/appuser/.ssh/id_rsa.pub`的内容。
```shell
vim /home/appuser/.ssh/id_rsa.pub
```
Public key of the development server:\
development服务器的公钥：
```shell
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDtvf8h2aotsTeNDTo/WVfo/XuuUhR+NpsOQI8JJ0+Xnvgz9eDNHxmHC1PJvA3e2j49Z3d9m4edMCI9fxvo8SLGh0nsWU2h5iwDQEn5ORAk7OwbziI2fu4wMCHa3zrAlpVinaqi0Y7ddghlCFz6a6JPYPUrTNjUDIOOwYndcJNOxEnzE+EX2Vlu1YoT3P/IWSEemW+Jj4qpk5Svjg3ETX6TvICU2WuOq4VI5X4eKNdgVzADiwJ2eWAWLZodWGlqPh4EUNx3hh3Ybt+Sdb0O14NtEwcao5Vg1VxX1n1w5WEXIfdcNobyvsqYPbZijUBQxe/HCERQXarsnx5B8/RizBqW1G6BTodWHUoaz6uzOEZrN3/qfOPzbDVvINphI83OZq/rCMUy5RqsvQXVOG1f3zl5JOL2AFu8BXorVELG9MG11ICEoigf/sMJddLEdBNIew7Io8ICS6bgaoh2YQleP+BYUDVwAHAtNaMmlIfBSVkQUaS7UmnW7Q0oJkUvjsFqwps= appuser@devserver
```

#### 2. Login to production server

```shell
# ssh -i cluster-key appuser@<PRODUCTION-SERVER-IP>
ssh -i /home/ubuntu/cluster-keys/cluster-key appuser@192.168.2.198
```

* Open file `/home/appuser/.ssh/authorized_keys` and paste the contents of the public key file in the `authorized_keys` file.\
打开文件`/home/appuser/.ssh/authorized_keys`，并将公钥文件的内容粘贴到`authorized_keys`文件中。
```shell
nano /home/appuser/.ssh/authorized_keys
vim .ssh/authorized_keys
```
    **Note:** Do not delete the public key of client machine, otherwise you will fail to log in production server from client server.

* Create a directory (it will be jump directory)\
创建一个目录(它将是跳转目录)
```shell
pwd
```
Output:
```shell
/home/appuser/
```
```shell
mkdir my_project
cd my_project
```

* Here is the path to your jump directory. Double check that user appuser is the owner of `my_project` directory.\
下面是跳转目录的路径。仔细检查用户appuser是`my_project`目录的所有者。
```shell
pwd
```
Output:
```shell
/home/appuser/my_project
```
Create git empty directory\
创建git空目录
```shell
git init --bare
```
```shell
Initialized empty Git repository in /home/appuser/my_project/
```
The command will initialize empty Git repository in `/home/appuser/my_project/`.\
该命令将在`/home/appuser/my_project/`中初始化一个空Git存储库。

* Create a git hook post-receive\
创建一个git hook post-receive
```shell
nano hooks/post-receive
vim hooks/post-receive
```
File contents:
```shell
#!/bin/bash
while read oldrev newrev ref
do
    if [[ $ref =~ .*/master$ ]];
    then
        echo "Master ref received. Deploying master branch to production..."
        sudo git --work-tree=/model_serving/ci_cd/production_server --git-dir=/home/appuser/my_project checkout -f
    else
        echo "Ref $ref successfully received. Doing nothing: only the master branch may be deployed on this server."
    fi
done
```

* Change permissions\
改变权限
```shell
# nano chmod +x hooks/post-receive
chmod +x hooks/post-receive
```
    * `chmod +x + <file_name>`: 给执行权限，字体变为粗体绿色。
    * `chmod -x + <file_name>`: 去除执行权限，字体变为常规黑色。

* Exit Production Server\
退出Production服务器

#### 3. Login to the Development Server

```shell
# ssh -i cluster-key appuser@<DEVELOPMENT-SERVER-IP>
ssh -i /home/ubuntu/cluster-keys/cluster-key appuser@192.168.2.80
```

* Create a directory
```shell
pwd
```
Output:
```
/home/appuser/
```
```shell
mkdir my_project
cd my_project
```

* This is your development directory. Double check that user appuser is the owner of `my_project` directory.\
这是您的development目录。仔细检查用户appuser是`my_project`目录的所有者。
```shell
pwd
```
Output:
```shell
/home/appuser/my_project
```

* Create empty git repository\
创建空的git库
```shell
git init
```
Output:
```shell
Initialized empty Git repository in /home/appuser/my_project/.git/
```

* Add files `model.h5` and `model.json` in `/home/appuser/my_project/` directory. The files are available in `model_serving/ci_cd/development_server/` directory.\
添加文件`model.h5`和`model.json`到`/home/appuser/my_project/`目录。这些文件可以在`model_serving/ci_cd/development_server/`目录中找到。
```shell
# Add files `model.h5` and `model.json`
scp -i /home/ubuntu/cluster-keys/cluster-key -r model_serving/ci_cd/development_server/model* appuser@192.168.2.80:/home/appuser/my_project
```

* Go to the `/home/appuser/my_project` directory\
进入`/home/appuser/my_project`目录
```shell
cd my_project
```

* Add files for the commit
```shell
git add .
```

* Commit files
```shell
git commit -m "new model"
```
```shell
# Please tell me who you are.
git config --global user.email "mengfei.liang.2445@student.uu.se"
git config --global user.name "MengFeiLiang"
```

* Connect development server's git to production server's git.\
连接development服务器的git到production服务器的git。
```shell
# git remote add production appuser@<PRODUCTIONS-SERVER-IP>:/home/appuser/my_project
git remote add production appuser@192.168.2.198:/home/appuser/my_project
```

* Push your commits to the production server\
将提交推到生产服务器
```shell
git push production master
```

In case you want to learn more about Git Hooks, visit the following link: [How To Use Git Hooks To Automate Development and Deployment Tasks](https://www.digitalocean.com/community/tutorials/how-to-use-git-hooks-to-automate-development-and-deployment-tasks)\
如果你想了解更多关于Git hook的知识，请访问以下链接：[如何使用Git hook自动化开发和部署任务](https://www.digitalocean.com/community/tutorials/how-to-use-git-hooks-to-automate-development-and-deployment-tasks)

#### 4. Go to `/model_serving/ci_cd/development_server/` directory. 

The directory contains the `neural_net.py` python script. The script will train a model and generate new model files `model.h5` and `model.json`. Open the training script `neural_net.py`, make changes in the model, install dependencies (`numpy` will be required) and run the script, move them in the git repository, commit changes and then push new model files in the running production pipeline.\
该目录包含`neural_net.py` python脚本。该脚本将训练一个模型并生成新的模型文件的`model.h5`和`model.json`。打开训练脚本`neural_net.py`，对模型进行更改，安装依赖项（需要`numpy`）并运行脚本，将它们移动到git存储库中，提交更改，然后在运行的生产管道中推入新的模型文件。

Install dependencies:
```shell
ssh -i /Users/lmf/PycharmProjects/MSc_DS/Important/key_DE.pem ubuntu@130.238.29.126
python3 --version
sudo apt install python3-pip
pip3 --version
pip3 install numpy
pip3 install keras
pip3 install tensorflow
```

Error:
```shell
Waiting for cache lock: Could not get lock /var/lib/dpkg
```
在这个时候，主要是因为apt还在运行，此时的解决方案如下：找到并且杀掉所有的`apt-get`和`apt`进程
```shell
ps -A | grep apt
sudo kill -9 <进程ID>
```

* Run the script\
运行脚本
```shell
cd model_serving/ci_cd/development_server
python3 neural_net.py
```
Output:
```python
Accuracy: 76.86
Saved model to disk
Loaded model from disk
[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 0 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 0 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 0 (expected 1)
```

* Copy model files (`model.h5` and `model.json`) from the development directory `model_serving/ci_cd/development_server` to the git repository `/home/appuser/project` on the development server.\
从development目录`model_serving/ci_cd/development_server/`复制模型文件（`model.h5`和`model.json`）到development服务器的git仓库`/home/appuser/project`里。
```shell
# cp model_serving/ci_cd/development_server/model* /home/appuser/my_project/.
scp -i /home/ubuntu/cluster-keys/cluster-key -r model_serving/ci_cd/development_server/model* appuser@192.168.2.80:/home/appuser/my_project/.
```

* Add to the git repository.
```shell
ssh -i /home/ubuntu/cluster-keys/cluster-key appuser@192.168.2.80
cd my_project
git add .
```

* Finally, commit and push the new model files.\
最后，提交并推入新的模型文件。
```shell
git commit -m "new model"
git push production master
```
Output:
```shell
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 1.22 KiB | 1.22 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Master ref received. Deploying master branch to production...
To 192.168.2.198:/home/appuser/my_project
   e2567cb..6be510c  master -> master
```

* Access the URL `http://<PRODUCTION-SERVER-IP>:5100/predictions` and you will see the changes in predictions.
    * http://130.238.28.163:5100/predictions

* Improve the model by changing parameters or network architecture in the `neural_net.py` file and repeat the step-3 to push the new model to the production pipeline.\
通过更改`neural_net.py`文件中的参数或网络架构来改进模型，并重复步骤3将新模型推到生产管道中。

#### Questions

##### 1. What are git hooks? Explain the `post-receive` script available in this task.
什么是git hooks？解释此任务中的`post-receive`脚本。

* [Git Hooks | Atlassian Git Tutorial](https://www.atlassian.com/git/tutorials/git-hooks)
* [8.3 Customizing Git - Git Hooks](https://git-scm.com/book/en/v2/Customizing-Git-Git-Hooks)

**Answer**

Git hooks are scripts that run automatically every time a particular event occurs in a Git repository. They let you customize Git's internal behavior and trigger customizable actions at key points in the development life cycle.\
Git hooks是每当Git存储库中发生特定事件时自动运行的脚本。它们允许您定制Git的内部行为，并在开发生命周期的关键时刻触发可定制的操作。

The `post-receive` hook runs after the entire process is completed and can be used to update other services or notify users.\
`post-receive` hook 在整个进程完成后运行，可以用来更新其他服务或通知用户。

For each ref being pushed, the three pieces of info (`old rev`, `new rev`, `ref`) will be fed to the script, separated by white space, as standard input. We can read this with a `while` loop to surround the `git` command.
So now, we will have three variables set based on what is being pushed. For a master branch push, the `ref` object will contain something that looks like `refs/heads/master`. We can check to see if the ref the server is receiving has this format by using an `if` construct.
We should add an `else` block to notify the user when a non-master branch was successfully received, even though the action won’t trigger a deploy.

##### 2. We have created an empty git repository on the production server. Why we need that directory?
我们已经在production服务器上创建了一个空的git存储库。为什么我们需要那个目录?

* [How To Use Git Hooks To Automate Development and Deployment Tasks](https://www.digitalocean.com/community/tutorials/how-to-use-git-hooks-to-automate-development-and-deployment-tasks)
* [Using Git for Deployment - UserFrosting Documentation](https://learn.userfrosting.com/going-live/vps-production-environment/git-for-deployment)

**Answer**

A bare repository does not have a working directory and is better for servers that you will not be working with much directly.\
裸存储库没有工作目录，对于您不会直接使用的服务器来说更好。

The reason we use a bare repository is because it separates the location of the repository (the files managed by git that live in the .git directory), and the working tree (the files you normally work with and from which you commit to the repository).\
我们使用裸存储库的原因是它分离了存储库的位置(位于.git目录中的git管理的文件)和工作树(您通常使用和向存储库提交的文件)。

If we used a non-bare repository, we wouldn't be able to "push to production" because git does not allow you to push to a checked-out branch. Since the checked-out branch would be the actual set of files that the webserver is running your application on, it would make it impossible to change these files when we do git push.\
如果我们使用一个非裸存储库，我们将不能“推送到生产”，因为git不允许你推送到一个签出的分支。因为签出分支将是运行你的应用程序的web服务器上的实际文件集，所以当我们做git push时，它将不可能改变这些文件。

##### 3. Write the names of four different git hooks and explain their functionalities.
写出四个不同git hooks的名称，并解释它们的功能。

* [Git Hooks | Atlassian Git Tutorial](https://www.atlassian.com/git/tutorials/git-hooks)

**Hints:** Read the link available in the task or access sample scripts available in the hooks directory.\
读取任务中可用的链接，或者访问hook目录中可用的示例脚本。

**Answer**

* `Pre-Commit`\
The `pre-commit` script is executed every time you run `git commit` before Git asks the developer for a commit message or generates a commit object. This hook is used to inspect the snapshot that is about to be committed.

* `Prepare Commit Message`\
The `prepare-commit-msg` hook is called after the `pre-commit` hook to populate the text editor with a commit message. This is a good place to alter the automatically generated commit messages for squashed or merged commits.

* `Commit Message`\
The `commit-msg` hook is much like the `prepare-commit-msg` hook, but it’s called after the user enters a commit message. This is an appropriate place to warn developers that their message doesn’t adhere to your team’s standards.

* `Post-Commit`\
The `post-commit` hook is called immediately after the `commit-msg` hook. It can not change the outcome of the `git commit` operation, so it’s used primarily for notification purposes.

##### 4. How deployment of multiple servers (in task 3) can be achieved with Kubernetes? Draw a diagram of the deployment highlighting the nodes, services, configuration map and secret.
如何使用Kubernetes实现多台服务器（任务3中）的部署？绘制一个突出显示节点、服务、配置映射和秘密的部署图。

How deployment of multiple servers can be achieved with Kubernetes? nodes, services, configuration map and secret.

##### 5. Write half a page summary about the task and add screenshots if needed.
写半页的任务总结，如果需要的话添加截图。